In [12]:
import scipy.io
from scipy.sparse import csc_matrix
import numpy as np
import pip
import os
import math

# pip.main(['install'] + ['nodejs'])
# pip install ipympl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


import ipympl
%matplotlib widget
# !pip install --upgrade jupyterlab
# !pip install brainrender
# !conda install nodejs
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !jupyter labextension install jupyter-matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

# BrainRender
import brainrender
from brainrender.scene import Scene
from vedo import Plotter
from brainrender.scene import Scene
# Rendering your scene in a separate window
from vedo import embedWindow
embedWindow(False)

In [ ]:
from scipy import io

probeLoc = io.loadmat('probeLocations.mat')
probeBorders = io.loadmat('probeBorders.mat', squeeze_me=True)

mouse_names = ['Krebs','Waksman','Robbins']
# start of spontaneous activity in each mouse (in seconds)
tstart = [3811, 3633, 3323];

## Function for extracting each mouse's parameters
def extract_data(k,imouse):
## k is the number of probe (0 to 7)
## imouse is the list containing mouse names (0 to 2)
    if (k in [i for i in range(8)]) and (imouse in [i for i in range(3)]):
# imouse = 0

        spks = io.loadmat('Spikes/spks%s_Feb18.mat'%mouse_names[imouse], squeeze_me=True)
        faces = io.loadmat('Faces/%s_face_proc.mat'%mouse_names[imouse], squeeze_me=True)

        # probe k
        # k = 0
        # spike times (in seconds)
        st = spks['spks'][k][0]   # (606058,)
        # clusters
        clu = spks['spks'][k][1]  # (606058,)
        # cluster heights (in microns) ==> Height of each cluster on the probe
        # (see siteCoords to convert to site location)
        Wh = spks['spks'][k][2]    # (127,)

        # processed faces
        motSVD = faces['motionSVD']
        video_timestamps = faces['times']

        # where is the probe in the brain (consolidated labels)
        # borders are in microns
        # use Wh to determine which clusters are in which brain region
        borders = probeBorders['probeBorders'][imouse]['borders'][k]
        for j in range(len(borders)):
           b = borders[j]
           print('upper border %d, lower border %d, area %s'%(b[0],b[1],b[2]))
           wneurons = np.logical_and(Wh>=b[1], Wh<b[0])
           nn = wneurons.sum()
           print('%d neurons in %s'%(nn,b[-1]))

        # where is the probe in the brain (in microns) ==> (384,3)
        ccfCoords = probeLoc['probeLocations'][0][imouse]['probe'][0][k]['ccfCoords']
        # name of area in Allen ontology by site on electrode ==> (384,1)
        ccfNames = probeLoc['probeLocations'][0][imouse]['probe'][0][k]['ccfOntology']
        # coordinates of each site on the electrode ==> (384,2)
        siteCoords = probeLoc['probeLocations'][0][imouse]['probe'][0][k]['siteCoords']
        
        # Initiate DataFrame for these data
        
    elif (k in [i for i in range(8)]) and (imouse not in [i for i in range(3)]):
        raise Exception('imouse should be the integer from 0 to 2 only.')
    elif (k not in [i for i in range(8)]) and (imouse in [i for i in range(3)]):
        raise Exception('i should be the integer from 0 to 7 only.')
    else:
        raise Exception('Both of your function inputs are not eligible as i needs to be integer from 0 to 7 and imouse needs to be integer from 0 to 2 only.')
    
    return st,clu,Wh,motSVD,video_timestamps,ccfCoords,ccfNames,siteCoords

# 0: st
# 1: clu
# 2: Wh
# 3: motSVD
# 4: video_timestamps
# 5: ccfCoords
# 6: ccfNames
# 7: siteCoords

def count_neurons(time_interval, file):
    t1,t2 = time_interval
    trial_spikes = file[0]
    interval = (trial_spikes > t1) & (trial_spikes < t2)
    trial_spikes = trial_spikes[interval]
    ids = file[1][interval]
    
    return np.unique(ids, return_counts = True)

def bin_into_array(file, bin_space):
    ids = file[1]
    idList = np.unique(ids)
    
    n_neurons = len(idList)
    n_neurons = n_neurons + 1
    n_bins = math.ceil((np.max(file[0])-np.min(file[0]))/(bin_space))
#     display(n_bins)
    data = np.zeros((n_neurons,n_bins))
#     display(data.shape)
    t_start = np.min(file[0])
    t_end = np.max(file[0])
    for b in range(n_bins):
        which_neurons, n_spikes = count_neurons((t_start + b*bin_space, t_start + (b+1)*bin_space),file)
#         display(which_neurons)
        data[which_neurons,b] = n_spikes
    return data

In [114]:
bin_space = 5
# bin_space = (np.max(Krebs[0])-np.min(Krebs[0]))/len(Krebs[4])
# Krebs = extract_data(5,0)
# which_neu, n_spikes = count_neurons((3830,3830.1),Krebs)
data = bin_into_array(Krebs, bin_space)
# display(which_neu)
# display(n_spikes)
# display(data[1])


In [115]:
fig = plt.figure()
# Provide the title for the spike raster plot
plt.title('Spike raster plot (5 s bins)')
# Give y axis label for the spike raster plot
plt.ylabel('Spike rate (Hz)')
# Give x axis label for the spike raster plot
plt.xlabel('Spontaneous activity time (seconds)')
# bin_space = (np.max(Krebs[0])-np.min(Krebs[0]))/len(Krebs[4])
display((np.max(Krebs[0])-np.min(Krebs[0]))/len(Krebs[4]))
t = np.array([np.arange(np.min(Krebs[0]),np.max(Krebs[0]),bin_space)])
display(data[0])
plt.plot(t[0],data[1], 'b')
plt.plot(t[0],data[2], 'r')
plt.plot(t[0],data[3], 'g')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.00632419322250231

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [77]:
# fig = plt.figure()
# plt.title('Motion Energy Principal Components')
# # Give y axis label for the spike raster plot
# plt.ylabel('Neuron')
# # Give x axis label for the spike raster plot
# plt.xlabel('Spontaneous activity time (seconds)')

# plt.plot(Krebs[4],Krebs[3][:,3], 'b')
# # plt.plot(Krebs[4],Krebs[3][:,2], 'r')
# # plt.plot(Krebs[4],Krebs[3][:,3], 'g')
# # plt.ylim(-1200,1200)
# plt.show()

display(Krebs[3][:,3])

array([-103.86932 , -103.86932 , -180.97568 , ...,  262.0697  ,
        -11.881396,   82.2555  ], dtype=float32)